In [10]:
%run preprocecssing_helpers.ipynb
import os
import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams.update({'font.size': 15})
# matplotlib.rcParams.

In [1]:
new_stim_code_event_ids = {
     
    "b-f2-wrd":1,
    "b-f1-wrd":2,
    "b-tg-wrd":3,

    "b-f2-spk":4,
    "b-f1-spk":5,
    "b-tg-spk":6,

    "b-f2-pic":7,
    "b-f1-pic":8,
    "b-tg-pic":9,

    "b-f2-snd":10,
    "b-f1-snd":11,
    "b-tg-snd":12,

    "f-f2-wrd":13,
    "f-f1-wrd":14,
    "f-tg-wrd":15,

    "f-f2-spk":16,
    "f-f1-spk":17,
    "f-tg-spk":18,

    "f-f2-pic":19,
    "f-f1-pic":20,
    "f-tg-pic":21,

    "f-f2-snd":22,
    "f-f1-snd":23,
    "f-tg-snd":24
}

In [95]:
# raw = mne.io.read_raw_eeglab(eeglab_files[2],preload = True)

In [2]:
def list_eeglab_files(root_path):
    """
    NOTE: must perform magic command %run preprocecssing_helpers.ipynb before using this method.
    
    Description: 
        Lists all eeglab .set files within a directory. 
        Assuming that the file format is: 
        Root Folder -> Subject Folder -> eeglab file contents
        example: 
        EEGLAB (root) -> SA20140220 (subject) -> 20140220_1009.set
        or 
        F:\\EEGLAB\\SA20140220\\20140220_1009.set
    
    Variables: 
        root_path: the folder containing all subject folders
    ----
    Returns:
        List of file .set path locations.
    """
    
    eeglab_files_list = os.listdir(root_path)
    eeglab_dict = search_folders(root_path, eeglab_files_list)
    eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
    
    return eeglab_files

In [97]:
# eeglab_files = list_eeglab_files(path)

In [5]:
def find_events(eeglab):
    """
    Description:
        returns the events of an eeglab .set object
        
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    ------
    returns: np array of 3d lists
    """
    return mne.find_events(eeglab, output= 'offset',
                              consecutive= True,
                              min_duration=1/eeglab.info['sfreq'], 
                              shortest_event= 1, 
                              initial_event=False)
    

In [99]:
# events = find_events(raw)
# type(events)

In [6]:
def get_trials(events):
    """
    Descriptions:
        returns a list of trials where each trial contains timestamp, offset and event_id
    
    Variables:
        events: A np array of 3d lists. use mne.find_events or custom find_events(eeglab) method already written.
    -----
    returns: np array of lists
    """
    trials = []
    current_trial = []
    for event in events:
        if event[-1] == 31 and current_trial != []:
            trials.append(current_trial)
            current_trial = []
            current_trial.append(event)
        else:
            current_trial.append(event)
            
    trials.append(current_trial) # the last trial does not have an indicator, so append it after the loop ends
    return trials

In [101]:
# trials = get_trials(events)

In [102]:
# trials[1][0]

In [7]:
def filter_bad_trials(eeglab):
    """
    Description:
        Filters out all events that are not within the 1-24 stim-code definitions.
        Also filters for trials that have a reaction time greater than .500ms as the subject 
        failed to click in time and was marked incorrect anyway.
        
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    ------
    returns:
        new_events: 
            np array of 3d list of events 
    """
    
    events = find_events(eeglab)
    trials = get_trials(events)
    button_clicks = [131,132]
    accuracy = [129,130]
    found_first_stim_code = False
    found_third_stim_code = False
    isEnd_of_trial = False
    new_events = []
    good_trials = []
    bad_trials_count = 0
    for trial in trials:
#         print(trial)
        for event in trial:
            # find first stim-code
            if event[-1] <= 24 and found_first_stim_code == False: 
                first_stim_code = event
                found_first_stim_code = True
#                 print("found first stim-code: {0}".format(first_stim_code))

            # find third stim-code
            if found_first_stim_code and event[-1] <= 24:
                #the third stim-code is one less than the first. 
                if event[-1] + 1 == first_stim_code[-1]:
                    third_stim_code = event
                    found_third_stim_code = True
            
            if event[-1] in button_clicks:
                button_click = event        
                
            if event[-1] in accuracy:
                #end of trial
                if event[-1] == 130: # incorrect
                    button_click = event
                    time_diff = (button_click[0] - third_stim_code[0]) * (1/512) #compute time in seconds
                    if time_diff > 1:
                        print("time difference {0}".format(time_diff))
                        print(third_stim_code)
                        print(button_click)
                        bad_trials_count+=1
                    else:
                        #store this trial
                         good_trials.append(trial)
                else:
                    good_trials.append(trial)
                found_first_stim_code = False
                found_third_stim_code = False
                
    print("\n **{0} bad trials dropped**".format(bad_trials_count))
    return np.concatenate([x for x in good_trials])

In [104]:
# filtered_events = filter_bad_trials(raw)

In [8]:
def events_with_stim_codes(events):
    """
    Description:
        Method will return an mne events ndarray containing only 
        stim-codes (event_ids 1-24).
    Variables:
        events: list of 3d lists 
    -----
    Returns:
        new_events list of 3d lists containing events of stim-codes    
    """
    new_events = []
    for event in events:
        if event[-1] <= 24:
            new_events.append(np.array(event))
            
    return np.array(new_events)
    

In [9]:
# some eeglab .set files may not have seen ALL of the 1-24 stim-code events.
# write method to return a proper event_id dictionary
def get_stim_code_event_ids(events, event_ids):
    """
    Description:
        return dictionay of event_ids that match the criteria of the eeglab file.
        Some files may not have all stim-code events, and to avoid errors when creating epochs, 
        this method will return a dictionary of compatible event_ids.
        
    Variables:
        events: ndarray of ints (n_events, 3)
        event_ids: dictionary of event_ids 
            ex. {'stim-code': 1}
    -----
    return: dictionary of event_ids
    """
    unique_events = np.unique(events[:,-1])
    event_id_values = event_ids.values()
    event_dict = {}
    
    for event_id in unique_events:
        if event_id in event_id_values:
            key = list(event_ids.keys())[event_id-1]
            event_dict[key] = event_id
            
    return event_dict

In [107]:
def artifact_removal(eeglab):
    """
    NOTE: use this method after creating epoch objects to speed up process!
    
    Description:
        Perform notch filter, frequenc filter, and ICA 
        
    Variables: 
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    returns:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object

    """
    
    eeglab.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
    eeglab.filter(5,100, fir_design='firwin')
    eeglab.resample(256, n_jobs=10)
    ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
    ica.fit(eeglab,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
    ica.detect_artifacts(eeglab,start_find=None, stop_find=None, ecg_ch=None, 
                 ecg_score_func='pearsonr', skew_criterion=None, 
                 kurt_criterion=None, var_criterion=None,add_nodes=None)
    
    return ica.apply(eeglab)

In [108]:
def create_epochs(eeglab, events, event_id):
    """
    Description: 
        create mne.epoch object from eeglab data.
    
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
        events: array of int, shape (n_events, 3)
        event_id: dictionary of events 
            Keys are name of event as string, value is event_id as an int.
    
    returns:
        mne.epoch object
            
    """
    tmax = (1/eeglab.info['sfreq']) * 499 # tmax 1/sfrq * desired epoch length
    return mne.Epochs(eeglab, 
                      events=events,
                      event_id = new_stim_code_event_ids, 
                      preload = True, 
                      tmin=0, 
                      tmax=tmax) 
    

In [109]:
# tmax = (1/raw.info['sfreq']) * 499 # tmax 1/sfrq * desired epoch length
# epoch = mne.Epochs(raw, events=new_events, event_id = new_stim_code_event_ids, preload = True, tmin=0, tmax=tmax) 

In [1]:
def remove_artifacts_and_create_epochs(eeglab):
    """
    Description:
        Handles the start to finish process of finding events, event_ids, 
        artifact removal, and creaing epoch object.
    
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    
    Returns:
        mne.epoch object
    """
    eeglab = mne.io.read_raw_eeglab(eeglab,preload = True)
    events = filter_bad_trials(eeglab)
    stim_code_events = events_with_stim_codes(events)
    stim_code_event_ids = get_stim_code_event_ids(stim_code_events,new_stim_code_event_ids)
    raw_ica = artifact_removal(eeglab= eeglab)
    
    return create_epochs(eeglab= raw_ica, events= stim_code_events, event_id= stim_code_event_ids)
    

['F:\\EEGLAB\\SA20131216\\20131216_1310.set',
 'F:\\EEGLAB\\SA20131216\\20131216_1441.set',
 'F:\\EEGLAB\\SA20140205\\20140205_1114.set',
 'F:\\EEGLAB\\SA20140205\\20140205_1230.set',
 'F:\\EEGLAB\\SA20140206\\20140206_1029.set',
 'F:\\EEGLAB\\SA20140206\\20140206_1151.set',
 'F:\\EEGLAB\\SA20140207\\20140207_1146.set',
 'F:\\EEGLAB\\SA20140207\\20140207_1219.set',
 'F:\\EEGLAB\\SA20140217\\20140217_1136.set',
 'F:\\EEGLAB\\SA20140217\\20140217_1211.set',
 'F:\\EEGLAB\\SA20140219\\20140219_1046.set',
 'F:\\EEGLAB\\SA20140219\\20140219_1205.set',
 'F:\\EEGLAB\\SA20140220\\20140220_1009.set',
 'F:\\EEGLAB\\SA20140220\\20140220_1123.set',
 'F:\\EEGLAB\\SA20140225\\20140225_1422.set',
 'F:\\EEGLAB\\SA20140225\\20140225_1552.set',
 'F:\\EEGLAB\\SA20140227A\\20140227_1058.set',
 'F:\\EEGLAB\\SA20140227A\\20140227_1220.set',
 'F:\\EEGLAB\\SA20140227B\\20140227_1410.set']

Reading F:\EEGLAB\SA20140206\20140206_1151.fdt
Reading 0 ... 831247  =      0.000 ...  1623.529 secs...


5977 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
time difference 1.80859375
[509034      0      4]
[509960      0    130]

 **1 bad trials dropped**
Setting up band-stop filter
Filter length of 3379 samples (6.600 sec) selected
Setting up band-pass filter from 5 - 1e+02 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 25.0 Hz
Filter length of 845 samples (1.650 sec) selected
5977 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   14.2s
[Parallel(n_jobs=10)]: Done 129 out of 129 | elapsed:   27.2s finished


5584 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 128
Fitting ICA took 162.5s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
1979 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1979 events and 500 original time points ...
1011 bad epochs dropped


In [115]:
# get_stim_code_event_ids(new_events, new_stim_code_event_ids)

In [116]:
# list(new_stim_code_event_ids.keys())

In [117]:
# new_stim_code_event_ids.values()

In [118]:
# new_events = events_with_stim_codes(filtered_events)
# np.unique(new_events[:,-1])

In [119]:
# for set_file in eeglab_files[0]:
#     try:
#         start = time.time()
#         raw = mne.io.read_raw_eeglab(set_file, preload=True)
#         raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
#         raw.filter(5,100, fir_design='firwin')
#         raw.resample(256, n_jobs=10)
#         ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
#         ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
#         ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#                      ecg_score_func='pearsonr', skew_criterion=None, 
#                      kurt_criterion=None, var_criterion=None,add_nodes=None)
#         raw_ica = ica.apply(raw)

#         m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 1, initial_event=True)
#         epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)
# #         epoch.save(set_file.split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)
#         end = time.time()
#         print("SUCESSFULLY LOADED time elapsed: {0}", (end - start))
#     except :
#         print("***Error converting file: {0}***".format(set_file))
        

In [120]:
# start = time.time()
# # raw = mne.io.read_raw_eeglab(, preload=True)
# raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
# raw.filter(5,100, fir_design='firwin')
# raw.resample(256, n_jobs=10)
# ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
# ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
# ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#              ecg_score_func='pearsonr', skew_criterion=None, 
#              kurt_criterion=None, var_criterion=None,add_nodes=None)
# raw_ica = ica.apply(raw)

# # m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 0.5, initial_event=True)
# epoch = mne.Epochs(raw, events=new_events, event_id =new_stim_code_event_ids , preload = True, tmin=0, tmax=1,)
# end = time.time()
# print("time elapsed: {0}", (end - start))

## check if saving and loading epoch file works.

In [121]:
# os.chdir(epoch_destination)

In [122]:

# epoch.save('20131216_1310_epo.fif',verbose = True)

In [123]:
# epoch_files = os.listdir(epoch_destination)

In [124]:
# epoch_file = epoch_files[0]
# epoch_ = mne.read_epochs(epoch_file,preload=True)

In [125]:
# epoch_.plot(title="epochs");

In [126]:
# for set_file in eeglab_files:
#     print(set_file.split('\\')[-1].split('.')[0] + "_epo.fif")

In [127]:
# # start = time.time()
# raw = mne.io.read_raw_eeglab(eeglab_files[1], preload=True)
# raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
# raw.filter(5,100, fir_design='firwin')
# raw.resample(256, n_jobs=10)
# ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
# ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
# ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#              ecg_score_func='pearsonr', skew_criterion=None, 
#              kurt_criterion=None, var_criterion=None,add_nodes=None)
# raw_ica = ica.apply(raw)

# m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 1.5, initial_event=True)
# epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)

## Read epoched data any plot

In [128]:
# path = 'E:\EpochedEEG'
# epoch_files = os.listdir(path)

In [129]:
# eeg_file = os.path.join(path,epoch_files[0])

In [130]:
# epoch = mne.read_epochs(eeg_file, preload = True)

In [131]:
# epoch.plot(n_channels=20, n_epochs = 10, title = eeg_file);